This notebook builds a table of summary statistics for the deep sequencing of antibody-selected HA libraries.

In [2]:
import pandas as pd
import numpy as np
import dms_tools.file_io
import dms_tools.utils

In [3]:
SRA_samples = {
    'SRR4841569':'L1_H17L19_c1_r1',
    'SRR4841570':'L1_H17L19_c2_r1',
    'SRR4841567':'L1_H17L19_c3_r1',
    'SRR4841568':'L1_H17L19_c1_r2',
    'SRR4841573':'L1_H17L19_c2_r2',
    'SRR4841574':'L1_H17L19_c3_r2',
    'SRR4841571':'L2_H17L19_c1',
    'SRR4841572':'L2_H17L19_c2',
    'SRR4841575':'L2_H17L19_c3',
    'SRR4841576':'L3_H17L19_c1',
    'SRR4841595':'L3_H17L19_c2',
    'SRR4841596':'L3_H17L19_c3',
    'SRR4841597':'L1_H17L10',
    'SRR4841598':'L2_H17L10',
    'SRR4841599':'L3_H17L10',
    'SRR4841600':'L1_H17L7',
    'SRR4841601':'L2_H17L7',
    'SRR4841602':'L3_H17L7',
    'SRR4841603':'L1_H18S415',
    'SRR4841604':'L2_H18S415',
    'SRR4841579':'L3_H18S415',
    'SRR4841578':'L1_mock_r1',
    'SRR4841581':'L1_mock_r2',
    'SRR4841580':'L2_mock',
    'SRR4841582':'L3_mock'}

samples = sorted(SRA_samples.values())

# pandas dataframe to compile sequencing statistics for each sample:
statsdf = pd.DataFrame(samples, columns=['sample'])

In [4]:
# The target bottleneck complexity determines how many barcoded molecules (per subamplicon) 
# are used as template in round 2 PCR.  'total' refers to this summed across all 6 subamplicons.
target_complexity = []
total_complexity = []
for s in samples:
    if ('H17L7' in s) or ('H17L10' in s) or ('H18S415' in s) or ('H17L19' in s):
        target = 150000
    elif ('L1_mock' in s):
        target = 700000
    elif (s == 'L2_mock' or s == 'L3_mock'):
        target = 500000
    target_complexity.append((s,target))
    total_complexity.append((s,target*6))
target_complexity = dict(target_complexity)
total_complexity = dict(total_complexity)
statsdf['targeted subamplicon barcode complexity'] = statsdf['sample'].map(target_complexity)
statsdf['targeted total barcode complexity'] = statsdf['sample'].map(total_complexity)

In [5]:
# Total read pairs obtained for each sample, total number of aligned barcodes for each sample
total_reads = []
total_aligned_barcodes = []

for s in samples:
    this_summarystats = pd.read_csv('counts_and_alignments/{0}_summarystats.txt'.format(s), 
                             delimiter='=', header=None, names=['description','number'])
    
    aligned_indecies = np.where(this_summarystats["description"].str.contains('aligned'))[0]
    total_read_index = np.where(this_summarystats["description"].str.contains('total read pairs'))[0]
    
    this_total_reads = this_summarystats.get_value(int(total_read_index),'number')
    this_total_aligned_barcodes = sum([this_summarystats.get_value(int(i),'number') for i in aligned_indecies])
    
    total_reads.append((s,this_total_reads))
    total_aligned_barcodes.append((s,this_total_aligned_barcodes))
    
total_reads = dict(total_reads)
total_aligned_barcodes = dict(total_aligned_barcodes)

statsdf['total reads'] = statsdf['sample'].map(total_reads)
statsdf['total aligned barcodes'] = statsdf['sample'].map(total_aligned_barcodes)

In [6]:
# Calculate median effective sequencing depth across all codons in HA, for each sample:
median_effective_depths = []

for s in samples:
    this_codoncounts = dms_tools.file_io.ReadDMSCounts('counts_and_alignments/{0}_counts.txt'.format(s), 'codon')
    depths = [this_codoncounts[key]['COUNTS'] for key in this_codoncounts.keys()]
    median_effective_depths.append((s,int(np.median(depths))))

median_effective_depths = dict(median_effective_depths)
statsdf['median effective depth'] = statsdf['sample'].map(median_effective_depths)

In [7]:
statsdf

,sample,targeted subamplicon barcode complexity,targeted total barcode complexity,total reads,total aligned barcodes,median effective depth
0,L1_H17L10,150000,900000,4041487,407110,66459
1,L1_H17L19_c1_r1,150000,900000,8695632,710204,114457
2,L1_H17L19_c1_r2,150000,900000,8624087,711932,111935
3,L1_H17L19_c2_r1,150000,900000,8741879,694883,108903
4,L1_H17L19_c2_r2,150000,900000,8511629,687655,108645
5,L1_H17L19_c3_r1,150000,900000,8192158,645638,103958
6,L1_H17L19_c3_r2,150000,900000,8442463,644335,101350
7,L1_H17L7,150000,900000,2897563,668184,110674
8,L1_H18S415,150000,900000,3639853,629643,101859
9,L1_mock_r1,700000,4200000,7533263,1601770,234020


In [8]:
# save LaTeX code for table
f = open('summarytable.tex', 'w')
f.write(statsdf.to_latex(index=False))
f.close()